## Bronze Layer: Raw Ingestion
- COPY INTO from staged Steam dataset
- Minimal transformations

In [ ]:
USE ROLE ROLE_TEAM_TRIPLEJ;
USE WAREHOUSE ANIMAL_TASK_WH;
USE DATABASE DB_TEAM_TRIPLEJ;

#### Create 3 Schemas: Bronze, Silver, Gold

In [ ]:
CREATE SCHEMA IF NOT EXISTS Bronze;
CREATE SCHEMA IF NOT EXISTS Silver;
CREATE SCHEMA IF NOT EXISTS Gold;

#### Import selected datasets into bronze layer

In [ ]:
-- Create Bronze Stage with CSV format
CREATE STAGE IF NOT EXISTS BRONZE_STAGE;
CREATE OR REPLACE FILE FORMAT FF_CSV_STANDARD
  TYPE = CSV
  PARSE_HEADER = TRUE
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  EMPTY_FIELD_AS_NULL = TRUE;

In [ ]:
-- Use Bronze Schema
USE SCHEMA BRONZE;

-- Create file format

-- Create Bronze Tables via Schema interface
CREATE OR REPLACE TABLE Bronze.steam_games
USING TEMPLATE (
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) WITHIN GROUP (ORDER BY order_id)
    FROM TABLE(
        INFER_SCHEMA(
            LOCATION => '@DB_TEAM_TRIPLEJ.BRONZE.BRONZE_STAGE/steam.csv',
            FILE_FORMAT => 'FF_CSV_STANDARD'
        )
    )
);

CREATE OR REPLACE TABLE Bronze.steam_descriptions
USING TEMPLATE (
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) WITHIN GROUP (ORDER BY order_id)
    FROM TABLE(
        INFER_SCHEMA(
              LOCATION => '@DB_TEAM_TRIPLEJ.BRONZE.BRONZE_STAGE/steam_description_data.csv',
              FILE_FORMAT => 'FF_CSV_STANDARD'
        )
    )
);

In [ ]:
-- Load Bronze
COPY INTO BRONZE.STEAM_GAMES
FROM @BRONZE_STAGE/steam.csv
FILE_FORMAT = (FORMAT_NAME = 'FF_CSV_STANDARD')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

COPY INTO BRONZE.STEAM_DESCRIPTIONS
FROM @BRONZE_STAGE/steam_description_data.csv
FILE_FORMAT = (FORMAT_NAME = 'FF_CSV_STANDARD')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

In [ ]:
-- Bronze Steam Games Verification
SELECT * FROM BRONZE.STEAM_GAMES LIMIT 10;

In [ ]:
-- Bronze Steam Descriptions Verification
SELECT * FROM BRONZE.STEAM_DESCRIPTIONS LIMIT 10;